In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
0,0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197,5호선 상일동,373.362877,NaN
1,12,28480888,옥돌영양탕,NaN,Q,음식,Q05,닭/오리요리,Q05A01,삼계탕전문,...,143220,4945.0,NaN,1,NaN,127.089673,37.557977,5호선 아차산(어린이대공원후문),354.921957,NaN
2,14,17175285,은진이네,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,121861,4206.0,NaN,NaN,NaN,126.956164,37.551340,5호선 애오개,53.083100,NaN
3,15,17175333,흥부보쌈,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,158721,7997.0,NaN,5,NaN,126.875987,37.529132,5호선 오목교(목동운동장앞),353.540138,NaN
4,28,20665661,가파도참치,NaN,Q,음식,Q03,일식/수산물,Q03A04,참치전문점,...,158050,8005.0,NaN,2,NaN,126.873242,37.524340,5호선 오목교(목동운동장앞),122.836352,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20044,135800,17212093,만랩커피,오금역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138800,5702.0,NaN,NaN,103,127.125353,37.502381,5호선 오금,124.273859,NaN
20045,135805,17218896,얼음과자,NaN,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,157857,7510.0,NaN,1,123,126.812418,37.577436,5호선 방화,27.430119,NaN
20046,135806,17218871,오이소서울,상일점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134090,5276.0,NaN,1,104,127.165574,37.550924,5호선 상일동,983.009054,NaN
20047,135807,17218873,얼음왕국명일14,6호점,Q,음식,Q07,패스트푸드,Q07A07,아이스크림판매,...,134780,5297.0,상가,1,106,127.146217,37.546317,5호선 굽은다리(강동구민회관앞),347.273163,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,NaN,상일동,상일동역 도전최강달인왕만두
1,옥돌영양탕,닭/오리요리,삼계탕전문,서울특별시 광진구 긴고랑로36길 24,5호선 아차산(어린이대공원후문),354.921957,NaN,아차산,아차산역 옥돌영양탕
2,은진이네,한식,한식/백반/한정식,서울특별시 마포구 마포대로 196,5호선 애오개,53.083100,NaN,애오개,애오개역 은진이네
3,흥부보쌈,한식,족발/보쌈전문,서울특별시 양천구 목동동로 309,5호선 오목교(목동운동장앞),353.540138,NaN,오목교,오목교역 흥부보쌈
4,가파도참치,일식/수산물,참치전문점,서울특별시 양천구 오목로54길 2,5호선 오목교(목동운동장앞),122.836352,NaN,오목교,오목교역 가파도참치
...,...,...,...,...,...,...,...,...,...
20044,만랩커피,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 중대로23길 21,5호선 오금,124.273859,NaN,오금,오금역 만랩커피
20045,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,NaN,방화,방화역 얼음과자
20046,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,NaN,상일동,상일동역 오이소서울
20047,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,NaN,굽은다리,굽은다리역 얼음왕국명일14


In [6]:
items = data["검색어"][5571:]
items

5571         굽은다리역 갸또마마
5572         광나루역 팔월잔칫날
5573         마포역 이자카야키쿠
5574      송정역 황소곱창구이공항점
5575            공덕역 호계식
              ...      
20044          오금역 만랩커피
20045          방화역 얼음과자
20046        상일동역 오이소서울
20047    굽은다리역 얼음왕국명일14
20048         상일동역 얼음과자
Name: 검색어, Length: 14478, dtype: object

In [7]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}


In [8]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path=r'C:\Users\정예림\Desktop\텍스트마이닝\chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('link_photo > img') #이미지
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url[0].get('src')}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")


진행상황 : 1/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/14478
식당 존재 x
진행상황 : 8/14478
식당 존재 x
진행상황 : 9/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/14478
식당 존재 x
진행상황 : 11/14478
식당 존재 x
진행상황 : 12/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 14/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 15/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/14478
식당 존재 x
진행상황 : 17/14478
식당 존재 x
진행상황 : 18/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 19/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 20/14478
식당 존재 x
진행상황 : 21/14478
식당 존재 x
진행상황 : 22/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 23/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 24/14478
식당 존재


식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 193/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 194/14478
식당 존재 x
진행상황 : 195/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 196/14478
식당 존재 x
진행상황 : 197/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 198/14478
식당 존재 x
진행상황 : 199/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 200/14478
식당 존재 x
진행상황 : 201/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 202/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 203/14478
식당 존재 x
진행상황 : 204/14478
식당 존재 x
진행상황 : 205/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 206/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 207/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 208/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 209/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 210/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 211/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 212/14478
식당 존재 x
진행상황 : 213/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 214/14478
식당 존재 x
진행상황 : 215/14478
식당 존재
5
더 이상 리뷰 존

식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 386/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 387/14478
식당 존재 x
진행상황 : 388/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 389/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 390/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 391/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 392/14478
식당 존재 x
진행상황 : 393/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 394/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 395/14478
식당 존재 x
진행상황 : 396/14478
식당 존재 x
진행상황 : 397/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 398/14478
식당 존재 x
진행상황 : 399/14478
식당 존재 x
진행상황 : 400/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 401/14478
식당 존재 x
진행상황 : 402/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 403/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 404/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 405/14478
식당 존재 x
진행상황 : 406/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 407/14478
식당 존재 x
진행상황 : 408/1447

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 574/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 575/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 576/14478
식당 존재 x
진행상황 : 577/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 578/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 579/14478
식당 존재 x
진행상황 : 580/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 581/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 582/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 583/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 584/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 585/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 586/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 587/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 588/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 589/14478
식당 존재 x
진행상황 : 590/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 591/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 592/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 593/1

식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 763/14478
식당 존재 x
진행상황 : 764/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 765/14478
식당 존재 x
진행상황 : 766/14478
식당 존재 x
진행상황 : 767/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 768/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 769/14478
식당 존재 x
진행상황 : 770/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 771/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 772/14478
식당 존재 x
진행상황 : 773/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 774/14478
식당 존재 x
진행상황 : 775/14478
식당 존재 x
진행상황 : 776/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 777/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 778/14478
식당 존재 x
진행상황 : 779/14478
식당 존재 x
진행상황 : 780/14478
식당 존재 x
진행상황 : 781/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 782/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 783/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 784/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 785/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 786/14478
식당 존재 x
진

...

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2020/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2021/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2022/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2023/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2024/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2025/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2026/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2027/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2028/14478
식당 존재 x
진행상황 : 2029/14478
식당 존재 x
진행상황 : 2030/14478
식당 존재 x
진행상황 : 2031/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2032/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2033/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2034/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2035/14478
식당 존재 x
진행상황 : 2036/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2037/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2038/14478
식당 존재 x
진행상황 : 2039/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2

식당 존재 x
진행상황 : 2199/14478
식당 존재 x
진행상황 : 2200/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2201/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2202/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2203/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2204/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2205/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2206/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2207/14478
식당 존재 x
진행상황 : 2208/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2209/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2210/14478
식당 존재 x
진행상황 : 2211/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2212/14478
식당 존재 x
진행상황 : 2213/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2214/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2215/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2216/14478
식당 존재 x
진행상황 : 2217/14478
식당 존재 x
진행상황 : 2218/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2219/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2220/14478
식당 존재
3
더 이

식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2376/14478
식당 존재 x
진행상황 : 2377/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2378/14478
식당 존재 x
진행상황 : 2379/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2380/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2381/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2382/14478
식당 존재 x
진행상황 : 2383/14478
식당 존재 x
진행상황 : 2384/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2385/14478
식당 존재 x
진행상황 : 2386/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2387/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2388/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2389/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2390/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2391/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2392/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2393/14478
식당 존재
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2394/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2395/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2396/14478
식당 존재 x
진행상

식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2559/14478
식당 존재 x
진행상황 : 2560/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2561/14478
식당 존재 x
진행상황 : 2562/14478
식당 존재 x
진행상황 : 2563/14478
식당 존재 x
진행상황 : 2564/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2565/14478
식당 존재 x
진행상황 : 2566/14478
식당 존재 x
진행상황 : 2567/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2568/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2569/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2570/14478
식당 존재 x
진행상황 : 2571/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2572/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2573/14478
식당 존재
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2574/14478
식당 존재 x
진행상황 : 2575/14478
식당 존재
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2576/14478
식당 존재
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2577/14478
식당 존재 x
진행상황 : 2578/14478
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2579/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2580/14478
식당 존재
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2581

KeyboardInterrupt: 

In [9]:
img_elem

,ItemID,img_url
0,굽은다리역 갸또마마,None
1,굽은다리역 갸또마마,None
2,굽은다리역 갸또마마,None
3,굽은다리역 갸또마마,None
4,굽은다리역 갸또마마,None
...,...,...
9913,화곡역 맛집,None
9914,화곡역 맛집,None
9915,화곡역 맛집,None
9916,화곡역 맛집,None


In [10]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,1enlhhg,굽은다리역 갸또마마,3점,2020.12.20.
1,19v4as4,굽은다리역 갸또마마,5점,2020.07.17.
2,g76mqt,굽은다리역 갸또마마,5점,2020.07.17.
3,e4kau9,굽은다리역 갸또마마,4점,2020.03.19.
4,e4rpa1,굽은다리역 갸또마마,5점,2020.02.04.
...,...,...,...,...
9913,None,화곡역 맛집,None,2019.03.06.
9914,None,화곡역 맛집,None,2018.12.12.
9915,None,화곡역 맛집,None,2016.09.19.
9916,None,화곡역 맛집,None,2016.04.02.


In [11]:
img_elem

,ItemID,img_url
0,굽은다리역 갸또마마,None
1,굽은다리역 갸또마마,None
2,굽은다리역 갸또마마,None
3,굽은다리역 갸또마마,None
4,굽은다리역 갸또마마,None
...,...,...
9913,화곡역 맛집,None
9914,화곡역 맛집,None
9915,화곡역 맛집,None
9916,화곡역 맛집,None


In [12]:
review_elem

,ItemID,review
0,굽은다리역 갸또마마,20.12.20 기준 올리브빵은 비추입니다. 사과빵 무난하고 크루아상 안먹어봤지만 ...
1,굽은다리역 갸또마마,"갸또마마 빵 안드신 분들은 다이어트 하시는건가요?? 진짜 맛있습니다,, 맛있어요 그..."
2,굽은다리역 갸또마마,저 여기 일주일에 한번은 꼭 가는듯요 명일역 근방 최고 빵 맛집임니다 믿고 가세용
3,굽은다리역 갸또마마,올리브빵 맛있네요. 평소 올리브 싫어해서 샌드위치 먹을 땐 다 빼고 먹고 피자에도 ...
4,굽은다리역 갸또마마,올리브치아바타 너무맛잇떵
...,...,...
9913,화곡역 맛집,고기 양이 적어보였는데 쌈이랑 먹으니까 딱 좋아요. 쌈 무한리필 최고 👍🏻👍🏻너무너...
9914,화곡역 맛집,좋음.쌈먹고싶을때. 젓갈맛있음
9915,화곡역 맛집,제육볶음 양이 너무 작아요. 제육2인분 + 우렁1인분 시켰는데 제육양 보고 깜짝 놀...
9916,화곡역 맛집,쌈밥집. . 우렁쌈 된장이 넘 짬... 된장국은 괸찮음... 보통이네요


In [13]:
img_elem.head()

,ItemID,img_url
0,굽은다리역 갸또마마,None
1,굽은다리역 갸또마마,None
2,굽은다리역 갸또마마,None
3,굽은다리역 갸또마마,None
4,굽은다리역 갸또마마,None


In [16]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID  ItemID Rating    Timestamp
9896  10i1tsl  화곡역 맛집     5점  2021.03.14.
9897  11b08qg  화곡역 맛집     5점  2020.12.23.
9898   dvjq73  화곡역 맛집     2점  2020.12.15.
9899   v32d49  화곡역 맛집     5점  2020.07.21.
9900  13e9ti9  화곡역 맛집     2점  2020.07.19.
9901   so7ta9  화곡역 맛집     5점  2020.07.05.
9902  151mkun  화곡역 맛집     1점  2020.05.10.
9903  149t5t2  화곡역 맛집     5점  2020.04.30.
9904  13ldaeo  화곡역 맛집     5점  2020.01.17.
9905   upn58s  화곡역 맛집     5점  2019.12.27.
9906   n291j0  화곡역 맛집     5점  2019.12.15.
9907  10n0htf  화곡역 맛집     5점  2019.11.30.
9908   upfkh4  화곡역 맛집     5점  2019.11.16.
9909   l8va46  화곡역 맛집     5점  2018.12.25.
9910   8sbm4l  화곡역 맛집     5점  2018.12.06.
9911   qq5f7o  화곡역 맛집     5점  2018.11.04.
9912     None  화곡역 맛집   None  2019.04.03.
9913     None  화곡역 맛집   None  2019.03.06.
9914     None  화곡역 맛집   None  2018.12.12.
9915     None  화곡역 맛집   None  2016.09.19.
9916     None  화곡역 맛집   None  2016.04.02.
9917     None  화곡역 맛집   None  2013.10.06.
      ItemID img_url
9896  화곡역 맛집 

In [17]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_rating(8156).csv')

In [18]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_User_ID(8156).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_img(8156).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_review(8156).csv')

In [19]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


1101

In [100]:
print (data.loc[data['검색어'] == "서대문역 아웃백스테이크하우스"])

             상호명 상권업종중분류명 상권업종소분류명             도로명주소       역명          거리  \
2527  아웃백스테이크하우스       양식  패밀리레스토랑  서울특별시 중구 통일로 114  5호선 서대문  122.856647   

      카카오url    역              검색어  
2527     NaN  서대문  서대문역 아웃백스테이크하우스  


IndexError: list index out of range